In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client langchain-google-genai


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [3]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credential.json', SCOPES)
            creds = flow.run_local_server(port=0, open_browser=False) # Modified to not open browser
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()

In [4]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'  
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


In [5]:
def read_inbox(service, max_results=5):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=max_results
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_id = msg['id']
        message = service.users().messages().get(
            userId='me',
            id=msg_id,
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = message['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📩 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))


In [14]:
read_inbox(service)


📩 New Email
From: Fatimi Bee <b230955@skit.ac.in>
Subject: regarding request

📩 New Email
From: Fatimi Bee <b230955@skit.ac.in>
Subject: 

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Re: Fwd: to chk gmail auth

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Re: Fwd: to chk gmail auth

📩 New Email
From: fatimi bee <fatimibee312@gmail.com>
Subject: Fwd: to chk gmail auth


In [7]:

import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env
load_dotenv()

# Read API key
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

# Create OpenAI client
client = OpenAI(api_key=api_key)

try:
    # List models available for this key
    models = client.models.list()

    print("Models available for this key:")
    for model in models.data:
        print(model.id)

except Exception as e:
    print("Error:", e)


Models available for this key:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano

In [8]:
! pip install langchain langgraph langchain-openai python-dotenv 


Defaulting to user installation because normal site-packages is not writeable


In [9]:
def load_llm():
    from dotenv import load_dotenv
    from langchain_google_genai import ChatGoogleGenerativeAI

    load_dotenv()
    return ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature =0)


In [16]:
def build_email_prompt(user_prompt: str):
    from langchain_core.prompts import (
        ChatPromptTemplate,
        SystemMessagePromptTemplate,
        HumanMessagePromptTemplate
    )

    system_text = f"""
You are an AI email assistant.

User instructions:
{user_prompt}

IMPORTANT RULES:
- Output STRICT JSON only
- Do NOT use markdown
- Do NOT add names or signatures

JSON FORMAT:
{{{{ 
  "action": "reply" or "ignore",
  "reply": "plain email reply text only"
}}}}
"""

    system_msg = SystemMessagePromptTemplate.from_template(system_text)

    human_msg = HumanMessagePromptTemplate.from_template(
        "Subject: {subject}\nFrom: {sender}\nBody: {body}"
    )

    return ChatPromptTemplate.from_messages([system_msg, human_msg])


In [11]:
import base64
from email.mime.text import MIMEText

def fetch_unread_emails(service, max_results=5):
    return service.users().messages().list(
        userId="me",
        labelIds=["UNREAD"],
        maxResults=max_results
    ).execute().get("messages", [])

def extract_email(service, msg_id):
    msg = service.users().messages().get(
        userId="me",
        id=msg_id,
        format="full"
    ).execute()

    headers = msg["payload"]["headers"]
    subject = sender = ""

    for h in headers:
        if h["name"] == "Subject":
            subject = h["value"]
        elif h["name"] == "From":
            sender = h["value"]

    body = ""
    payload = msg["payload"]

    if "parts" in payload:
        for part in payload["parts"]:
            if part["mimeType"] == "text/plain":
                data = part["body"].get("data")
                if data:
                    body = base64.urlsafe_b64decode(data).decode()
    else:
        data = payload["body"].get("data")
        if data:
            body = base64.urlsafe_b64decode(data).decode()

    return subject, sender, body, msg["threadId"]

import re

def clean_email_body(body: str) -> str:
    # Remove markdown JSON blocks
    body = re.sub(r"```json[\s\S]*?```", "", body, flags=re.IGNORECASE)

    # Remove quoted replies (Gmail style)
    body = re.split(r"\nOn .* wrote:", body)[0]

    # Remove lines starting with >
    body = "\n".join(
        line for line in body.splitlines()
        if not line.strip().startswith(">")
    )

    return body.strip()

def send_reply(service, to, subject, reply, thread_id):
    message = MIMEText(reply)
    message["to"] = to
    message["subject"] = "Re: " + subject

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()

    service.users().messages().send(
        userId="me",
        body={"raw": raw, "threadId": thread_id}
    ).execute()

def mark_read(service, msg_id):
    service.users().messages().modify(
        userId="me",
        id=msg_id,
        body={"removeLabelIds": ["UNREAD"]}
    ).execute()


In [12]:
import json

def process_all_unread_emails(service, user_prompt):
    # Build prompt once
    prompt = build_email_prompt(user_prompt)
    llm = load_llm()

    # Fetch unread emails
    emails = fetch_unread_emails(service, max_results=20)

    print(f"\n📬 Total unread emails: {len(emails)}\n")

    if not emails:
        print("🎉 No unread emails.")
        return

    for idx, msg in enumerate(emails, start=1):
        print("=" * 70)
        print(f"📧 Processing email {idx}/{len(emails)}")

        subject, sender, body, thread_id = extract_email(service, msg["id"])
        body = clean_email_body(body)

        print(f"From   : {sender}")
        print(f"Subject: {subject}")
        print("Body preview:")
        print(body[:300])
        print("=" * 70)

        # ---- AI RESPONSE ----
        ai_response = llm.invoke(
            prompt.format_prompt(
                subject=subject,
                sender=sender,
                body=body
            ).to_messages()
        ).content

        # Parse JSON safely
        try:
            result = json.loads(ai_response)
        except json.JSONDecodeError:
            print("❌ AI returned invalid JSON. Skipping email.")
            continue

        action = result.get("action")
        reply = result.get("reply", "")

        if action == "ignore":
            print("🚫 AI chose to ignore this email.")
            continue

        print("\n🤖 AI Draft Reply")
        print("-" * 50)
        print(reply)
        print("-" * 50)

        decision = input("Send this reply? (y / edit / skip): ").lower()

        if decision == "y":
            send_reply(service, sender, subject, reply, thread_id)
            mark_read(service, msg["id"])
            print("✅ Reply sent & marked as read")

        elif decision == "edit":
            edited = input("\n✏️ Enter your edited reply:\n")
            send_reply(service, sender, subject, edited, thread_id)
            mark_read(service, msg["id"])
            print("✅ Edited reply sent & marked as read")

        else:
            print("⏭️ Skipped (email remains unread)")


In [17]:
USER_PROMPT = """
Reply only to genuine emails.
Ignore promotions and job ads.
Be polite, concise, and human-like.
Do not add name or signature.
"""

process_all_unread_emails(service, USER_PROMPT)



📬 Total unread emails: 1

📧 Processing email 1/1
From   : Fatimi Bee <b230955@skit.ac.in>
Subject: regarding request
Body preview:
Dear Sir/Madam,

I hope this email finds you well.

I am writing to inform you about my request regarding the mentioned
subject. I kindly request you to please look into this matter and guide me
on the next steps.

Please let me know if any further information is required from my side. I
will be hap

🤖 AI Draft Reply
--------------------------------------------------
Thank you for reaching out. To help you further, could you please provide more details about your request? For example, what is the subject of your request or when was it submitted?
--------------------------------------------------
✅ Reply sent & marked as read
